In [1]:
import pandas as pd
# import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestCentroid
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.style.use('ggplot')

In [2]:
def get_features():
    features = ['SEVERITYCODE', 'ADDRTYPE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT',
               'VEHCOUNT', 'JUNCTIONTYPE', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
               'SPEEDING', 'HITPARKEDCAR']
    return features

In [3]:
# The code was removed by Watson Studio for sharing.

SyntaxError: invalid syntax (<ipython-input-3-a86f5e5768a9>, line 2)

In [ ]:
def get_column_na_alt(col):
    fill_na_alt = {
        'SEVERITYCODE': 0,
        'ADDRTYPE': '',
        'INTKEY': 0,
        'COLLISIONTYPE': '',
        'PERSONCOUNT': 0,
        'PEDCOUNT': 0,
        'PEDCYLCOUNT': 0,
        'VEHCOUNT': 0,
        'JUNCTIONTYPE': '',
        'INATTENTIONIND': 'N',
        'UNDERINFL': '',
        'WEATHER': '',
        'ROADCOND': '',
        'LIGHTCOND': '',
        'SPEEDING': 'N',
        'HITPARKEDCAR': ''
    }
    return fill_na_alt[col]

In [ ]:
def get_processed_dataframe(df, columns):
    column_map = {}
    features = ['ADDRTYPE', 'COLLISIONTYPE', 'PERSONCOUNT', 'PEDCOUNT', 'PEDCYLCOUNT',
                'VEHCOUNT', 'JUNCTIONTYPE', 'INATTENTIONIND', 'UNDERINFL', 'WEATHER', 'ROADCOND', 'LIGHTCOND',
                'SPEEDING', 'HITPARKEDCAR']
    target = 'SEVERITYCODE'
    categorical_columns = ['COLLISIONTYPE', 'JUNCTIONTYPE', 'WEATHER', 'ROADCOND', 'LIGHTCOND']
    one_hot_columns = ['ADDRTYPE', 'INATTENTIONIND', 'UNDERINFL', 'SPEEDING', 'HITPARKEDCAR']
    for col in categorical_columns:
        df.fillna(get_column_na_alt(col), inplace=True)
        column_map[col] = {}
        unique_values = df[col].unique()
        for i, uv in enumerate(unique_values):
            column_map[col][uv] = i
    column_map['UNDERINFL'] = {'0': 'N',
                               '1': 'Y'}
    df.replace(column_map, inplace=True)
    for col in one_hot_columns:
        df_temp = pd.get_dummies(df[col])
        df_temp_col = df_temp.columns
        df_col_rename = {}
        for col_r in df_temp_col:
            df_col_rename[col_r] = col + '_' + col_r.upper()
        df_temp.rename(columns= df_col_rename, inplace=True)
        df = df.join(df_temp)
        if '' in df_col_rename.keys():
            df.drop([col, str(col+'_').strip()], axis=1, inplace=True)
        else:
            df.drop([col], axis=1, inplace=True)
    return df, column_map, features, target

In [ ]:
df_collision, column_map, features, target = get_processed_dataframe(df_collision, features)
df_collision.head()

In [ ]:
def get_column_correlation(df):
    correlation = df.corr()
    sbs.heatmap(correlation, xticklabels=correlation.columns, yticklabels=correlation.columns)
    return correlation

In [ ]:
correlation = get_column_correlation(df_collision)

In [ ]:
def apply_model(X_train, X_test, y_train, y_test, model):
    classifier = model
    classifier_model = classifier.fit(X_train, y_train)
    y_pred = classifier_model.predict(X_test)
    return get_accuracy_for_model(y_pred, y_test)


def get_accuracy_for_model(y_pred, y_test):
    return metrics.accuracy_score(y_test, y_pred)

In [ ]:
def apply_model_and_get_accuracy(df):
    X = df.drop('SEVERITYCODE', axis=1)
    y = df.SEVERITYCODE
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
    model_list = [
        {
            'name': 'Decision Tree',
            'function': DecisionTreeClassifier(),
            'accuracy': 0
        },
        {
            'name': 'Gaussian Naive Bayes',
            'function': GaussianNB(),
            'accuracy': 0
        },
        {
            'name': 'Nearest Neighbors',
            'function': NearestCentroid(),
            'accuracy': 0
        },
        {
            'name': 'Neural Network',
            'function': MLPClassifier(
                solver='lbfgs',
                alpha=1e-5,
                hidden_layer_sizes=(5, 2),
                random_state=1
            ),
            'accuracy': 0
        }
    ]
    for model in model_list:
        model['accuracy'] = apply_model(X_train, X_test, y_train, y_test, model['function'])
        print('Accuracy for ' + model['name'] + ': ', model['accuracy'])
    return model_list

In [ ]:
model_list = apply_model_and_get_accuracy(df_collision)


In [ ]:
accuracy_list = []
models_name = []
for model_data in model_list:
    accuracy_list.append(model_data['accuracy'])
    models_name.append(model_data['name'])
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(models_name,accuracy_list)
plt.show()